In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/e0c252b08cb179f9852573620061603e!OpenDocument#General"

In [3]:
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

# Scrape core courses

## 1. Collect all course codes and names

In [4]:
mech_start = page_soup.find("font", text="Bachelor of Applied Science in Mechanical Engineering")
mech_start.text

'Bachelor of Applied Science in Mechanical Engineering'

In [5]:
mech_start.findNext().findNext().findNext().findNext().findNext().text

''

In [6]:
#get all course codes and names for MechEng core courses
container = mech_start
course_names = []
course_codes = []
counter = 0

while container.text != "Bachelor of Applied Science in Mechanical Engineering with Aerospace Option":
    if bool(re.match("[A-Z]{4}-[0-9]{4}. [A-Za-z -]+", container.text)):
        course_names.append(container.text.split(". ")[1])
        course_codes.append(container.text.split(". ")[0])
        counter += 1
        
    container = container.findNext()
    
print(counter)

42


In [7]:
course_codes

['MATH-1270',
 'MATH-1720',
 'GENG-1110',
 'GENG-1330',
 'GENG-1180',
 'CHEM-1103',
 'MATH-1730',
 'PHYS-1410',
 'GENG-1200',
 'GENG-1190',
 'MATH-2790',
 'GENG-2320',
 'GENG-2340',
 'MECH-2210',
 'GENG-2500',
 'INDE-2010',
 'MATH-2780',
 'GENG-2190',
 'GENG-2220',
 'GENG-2200',
 'MECH-2230',
 'GENG-2180',
 'GENG-2980',
 'MECH-3212',
 'MECH-3233',
 'GENG-3130',
 'MECH-3211',
 'MECH-3223',
 'GENG-3980',
 'MECH-3217',
 'MECH-3220',
 'MECH-3228',
 'MECH-4221',
 'MECH-4259',
 'GENG-4980',
 'MECH-4218',
 'MECH-4200',
 'MECH-4211',
 'MECH-3221',
 'MECH-3224',
 'GENG-4210',
 'MECH-4200']

In [8]:
course_names

['Linear Algebra',
 'Differential Calculus',
 'Engineering Mechanics I',
 'Engineering and Design',
 'Engineering and the Profession',
 'Topics in General Chemistry',
 'Integral Calculus',
 'Introductory Physics II',
 'Engineering Thermofluids',
 'Technical Communications',
 'Differential Equations',
 'Engineering Software Fundamentals ',
 'Electrical and Computing Fundamentals ',
 'Dynamics',
 'Engineering and the Environment',
 'Engineering Management and Globalization',
 'Vector Calculus ',
 'Introduction Engineering Materials',
 'Treatment of Experimental Data',
 'Numerical Analysis for Engineering ',
 'Advanced Engineering and Design',
 'Mechanics of Deformable Bodies ',
 'Work Term I',
 'Thermodynamics',
 'Fluid Mechanics I',
 'Engineering Economics ',
 'Stress Analysis',
 'Machine Dynamics',
 'Work Term II',
 'Applied Thermodynamics',
 'Fluid Mechanics II',
 'Heat Transfer',
 'Machine Design ',
 'Computer Aided Engineering - CAE',
 'Work Term III',
 'Thermofluid Systems Design',

In [9]:
url_dict = {
    
    "MATH": "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/c4d12a22771cfa81852585310007c4bd!OpenDocument",
    "MECH": "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/9b72ca039c14a10585257364004c63b8!OpenDocument",
    "GENG": "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/9879b7f56ea6fde68525776d00585601!OpenDocument",
    "CHEM": "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/523d8435ae5f886785257364004a511c!OpenDocument",
    "PHYS": "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/fac10bc71a59714285257364004e518b!OpenDocument",
    "INDE": "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/9b72ca039c14a10585257364004c63b8!OpenDocument"
}

## 2. Test run - try to find the first course and scrape it

In [10]:
uClient = ureq(url_dict["MATH"])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

containers = page_soup.findAll("font")
for container in containers:
    if container.text.split(".")[0].strip() == "MATH-1720":
        break
        
container

<font face="Arial" size="2">MATH-1720.</font>

In [11]:
container.findNext().findNext().findNext().findNext()

<font face="Arial" size="2">This course will cover trigonometric functions and identities, inverse trigonometric functions, limits and continuity, derivatives and applications, mean value theorem, indeterminate forms and l’Hôpital’s rule, antiderivatives and an introduction to definite integrals. This course is for students who have taken both Ontario Grade 12 Advanced Functions (MHF4U) and Ontario Grade 12 Calculus and Vectors (MCV4U). Students who do not have credit for MCV4U should take MATH-1760. (Prerequisites: Ontario Grade 12 Advanced Functions (MHF4U) and Ontario Grade 12 Calculus and Vectors (MCV4U) or MATH-1780.) (Antirequisite: MATH-1760.) (3 lecture hours, 2 tutorial hours per week.)</font>

In [12]:
containers.index(container)

44

## 3. find the starting point of each course description, then compile (some descriptions are broken over sever containers...) and extract

In [13]:
#get the course descriptions for all MechEng Core courses
course_descs = []
counter = 0

for code in course_codes:
    
    subject = code.split("-")[0]
    uClient = ureq(url_dict[subject])
    time.sleep(1)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    
    containers = page_soup.findAll("font")
    for container in containers:
        if container.text.split(".")[0].strip() == code:
            course_desc = ""
            i = containers.index(container) + 1
            while i < len(containers) and bool(re.match("[A-Z]{4}-[0-9]{4}", containers[i].text.strip())) == False:
                if (course_names[course_codes.index(code)] not in containers[i].text or len(containers[i].text)>len(course_names[course_codes.index(code)])+15) and len(containers[i].text) > 0:
                    course_desc += containers[i].text.strip()
                i += 1
                
            course_descs.append(course_desc)
            print("scraped ", code)
            counter += 1
            break
            
counter

scraped  MATH-1270
scraped  MATH-1720
scraped  GENG-1110
scraped  GENG-1330
scraped  GENG-1180
scraped  CHEM-1103
scraped  MATH-1730
scraped  PHYS-1410
scraped  GENG-1200
scraped  GENG-1190
scraped  MATH-2790
scraped  GENG-2320
scraped  GENG-2340
scraped  MECH-2210
scraped  GENG-2500
scraped  INDE-2010
scraped  MATH-2780
scraped  GENG-2190
scraped  GENG-2220
scraped  GENG-2200
scraped  MECH-2230
scraped  GENG-2180
scraped  GENG-2980
scraped  MECH-3212
scraped  MECH-3233
scraped  GENG-3130
scraped  MECH-3211
scraped  MECH-3223
scraped  GENG-3980
scraped  MECH-3217
scraped  MECH-3220
scraped  MECH-3228
scraped  MECH-4221
scraped  MECH-4259
scraped  GENG-4980
scraped  MECH-4218
scraped  MECH-4200
scraped  MECH-4211
scraped  MECH-3221
scraped  MECH-3224
scraped  GENG-4210
scraped  MECH-4200


42

## 4. Inspect, clean, and store in dataframe

In [14]:
course_descs

['This course will coverlinear systems,linear transformations, matrix algebra, determinants, vectors in Rn, dot product, orthogonalization,diagonalization,eigenvectorsand eigenvalues,in the context of and with an emphasis on a broad range of applications in Science and Engineering. (Prerequisite:',
 'This course will cover trigonometric functions and identities, inverse trigonometric functions, limits and continuity, derivatives and applications, mean value theorem, indeterminate forms and l’Hôpital’s rule, antiderivatives and an introduction to definite integrals. This course is for students who have taken both Ontario Grade 12 Advanced Functions (MHF4U) and Ontario Grade 12 Calculus and Vectors (MCV4U). Students who do not have credit for MCV4U should take MATH-1760. (Prerequisites: Ontario Grade 12 Advanced Functions (MHF4U) and Ontario Grade 12 Calculus and Vectors (MCV4U) or MATH-1780.) (Antirequisite: MATH-1760.) (3 lecture hours, 2 tutorial hours per week.)',
 'Statics of partic

In [15]:
course_descs[21] = course_descs[21][20:]

In [16]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,MATH-1270,Linear Algebra,"This course will coverlinear systems,linear tr..."
1,MATH-1720,Differential Calculus,This course will cover trigonometric functions...
2,GENG-1110,Engineering Mechanics I,Statics of particles and rigid bodies; trusses...
3,GENG-1330,Engineering and Design,Introductory engineering design course. Visual...
4,GENG-1180,Engineering and the Profession,The Engineering and the Profession course is a...
5,CHEM-1103,Topics in General Chemistry,An introduction to selected topics in modern c...
6,MATH-1730,Integral Calculus,"This course will cover antiderivatives, the de..."
7,PHYS-1410,Introductory Physics II,"Wave motion, sound, electricity and magnetism,..."
8,GENG-1200,Engineering Thermofluids,"Introductory thermodynamics, fluid mechanics, ..."
9,GENG-1190,Technical Communications,The Technical Communications course focuses on...


In [17]:
df = df.drop([36], axis = 0) #duplicate course
df

,Course Number,Course Name,Course Description
0,MATH-1270,Linear Algebra,"This course will coverlinear systems,linear tr..."
1,MATH-1720,Differential Calculus,This course will cover trigonometric functions...
2,GENG-1110,Engineering Mechanics I,Statics of particles and rigid bodies; trusses...
3,GENG-1330,Engineering and Design,Introductory engineering design course. Visual...
4,GENG-1180,Engineering and the Profession,The Engineering and the Profession course is a...
5,CHEM-1103,Topics in General Chemistry,An introduction to selected topics in modern c...
6,MATH-1730,Integral Calculus,"This course will cover antiderivatives, the de..."
7,PHYS-1410,Introductory Physics II,"Wave motion, sound, electricity and magnetism,..."
8,GENG-1200,Engineering Thermofluids,"Introductory thermodynamics, fluid mechanics, ..."
9,GENG-1190,Technical Communications,The Technical Communications course focuses on...


# Scrape elective courses

In [18]:
#scrape the rest of mech courses as they are elective courses

url = "http://web4.uwindsor.ca/units/registrar/calendars/undergraduate/fall2020.nsf/982f0e5f06b5c9a285256d6e006cff78/9b72ca039c14a10585257364004c63b8!OpenDocument"

uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

## 1. scrape all mech courses that haven't been scraped yet

In [19]:
course_names2 = []
course_codes2 = []
course_descs2 = []
counter = 0
look_for_desc = False

containers = page_soup.findAll("font")

for container in containers:
    if not look_for_desc and bool(re.match("MECH-[24683]{2}[0-9]{2}\.\D", container.text)):
        current_code = container.text.split(".")[0].strip()
        if current_code not in course_codes:
            course_codes2.append(current_code)
            course_names2.append(container.text.split(".")[1].strip())
            look_for_desc = True
            print(course_codes2[-1])

    elif look_for_desc:
            course_desc2 = ""
            i = containers.index(container)
            while i < len(containers) and bool(re.match("MECH-[0-9]{4}", containers[i].text.strip())) == False and containers[i].text not in "MECHANICAL: APPROVED COURSES TO FULFILL NON-SPECIFIED ENGINEERING COURSE REQUIREMENTS AEROSPACE ENGINEERING AUTOMOTIVE ENGINEERING ENGINEERING MATERIALS INDUSTRIAL AND MANUFACTURING SYSTEMS ENGINEERING":
                if len(containers[i].text) > 0:
                    course_desc2 += containers[i].text.strip()
                i += 1
                
            course_descs2.append(course_desc2)
            counter += 1
            look_for_desc = False
        
counter

MECH-3215
MECH-4212
MECH-4228
MECH-4240
MECH-4241
MECH-4250
MECH-4251
MECH-4253
MECH-4255
MECH-3430
MECH-4440
MECH-4441
MECH-4463
MECH-4465
MECH-4467
MECH-3830
MECH-3831
MECH-4820
MECH-4821
MECH-4832
MECH-4834
MECH-4840
MECH-4841
MECH-4850


24

In [20]:
course_descs2

['Free, damped, and forced vibration of single and multi-degree of freedom systems with discrete masses. Exact and approximate methods of solution. Vibration isolation, vibration transducers, use of computers in vibration analysis. (Prerequisite: MECH-2210 and Semester 6 or higher standing.) (3 lecture, 2 tutorial hours a week.)',
 'Review of electromechanical components.  Practical application of microcontrollers in electromechanical systems.  Use of infrared sensors, photoresistors, operational amplifiers, timers, servomotors, and analog/digital converters in mechatronics systems.   A hands-on, laboratory-based course.(Prerequisite:',
 'Environmental impact assessment. Biophysical and socioeconomic impacts from engineering activities, processes, and projects. Human health and environmental risk concepts. Introduction to life cycle analysis, corporate/industrial environmental management, and environmental management systems. Students will undertake various project related and problem-

In [21]:
len(course_descs2)

24

## 2. Clean and store in dataframe

In [22]:
df2 = pd.DataFrame({
    
    "Course Number": course_codes2,
    "Course Name": course_names2,
    "Course Description": course_descs2   
    
})

df2

,Course Number,Course Name,Course Description
0,MECH-3215,Mechanical Vibrations,"Free, damped, and forced vibration of single a..."
1,MECH-4212,Mechatronics,Review of electromechanical components. Pract...
2,MECH-4228,Sustainability in Engineering,Environmental impact assessment. Biophysical a...
3,MECH-4240,Topics in Mechanical Engineering,Selected topics of current interest in Mechani...
4,MECH-4241,Directed Studies in Mechanical Engineering,A special course of studies in Mechanical Engi...
5,MECH-4250,Gas Dynamics,Basic concepts and one-dimensional flow equati...
6,MECH-4251,Turbomachines,Dimensional analysis and similitude; definitio...
7,MECH-4253,,"Heating, Ventilation, andAir ConditioningPrinc..."
8,MECH-4255,Environmental Effects and Control of Noise,"Physical properties of sound and noise, measur..."
9,MECH-3430,Automotive Engineering Fundamentals,Overview of primaryautomotive systems. Engine ...


In [23]:
df2["Course Description"][7] = df2["Course Description"][7][41:]

In [24]:
df2["Course Name"][7] = "Heating, Ventilation, and Air Conditioning"

In [25]:
df2

,Course Number,Course Name,Course Description
0,MECH-3215,Mechanical Vibrations,"Free, damped, and forced vibration of single a..."
1,MECH-4212,Mechatronics,Review of electromechanical components. Pract...
2,MECH-4228,Sustainability in Engineering,Environmental impact assessment. Biophysical a...
3,MECH-4240,Topics in Mechanical Engineering,Selected topics of current interest in Mechani...
4,MECH-4241,Directed Studies in Mechanical Engineering,A special course of studies in Mechanical Engi...
5,MECH-4250,Gas Dynamics,Basic concepts and one-dimensional flow equati...
6,MECH-4251,Turbomachines,Dimensional analysis and similitude; definitio...
7,MECH-4253,"Heating, Ventilation, and Air Conditioning",inciples of environmental air quality and occu...
8,MECH-4255,Environmental Effects and Control of Noise,"Physical properties of sound and noise, measur..."
9,MECH-3430,Automotive Engineering Fundamentals,Overview of primaryautomotive systems. Engine ...


## 3. Scrape some missing courses

In [26]:
#scrape the 36xx and 46xx courses - they are coded differently than the other elective courses
course_names3 = []
course_codes3 = []
course_descs3 = []
counter = 0
look_for_desc = False

containers = page_soup.findAll("font")

for container in containers:
    if not look_for_desc and bool(re.match("MECH-[346]{2}[0-9]{2}", container.text)):
        current_code = container.text.strip()
        if current_code not in course_codes and current_code not in course_codes2:
            course_codes3.append(current_code)
            try:
                course_names3.append(container.findNext("font").text.split(".")[1].strip())
            except IndexError:
                course_names3.append("")
            look_for_desc = True
            print(course_codes3[-1])

    elif look_for_desc:
            course_desc3 = ""
            i = containers.index(container) + 1
            while i < len(containers) and bool(re.match("MECH-[0-9]{4}", containers[i].text.strip())) == False and containers[i].text not in "MECHANICAL: APPROVED COURSES TO FULFILL NON-SPECIFIED ENGINEERING COURSE REQUIREMENTS AEROSPACE ENGINEERING AUTOMOTIVE ENGINEERING ENGINEERING MATERIALS INDUSTRIAL AND MANUFACTURING SYSTEMS ENGINEERING":
                if len(containers[i].text) > 0:
                    course_desc3 += containers[i].text.strip()
                i += 1
                
            course_descs3.append(course_desc3)
            counter += 1
            look_for_desc = False
        
counter

MECH-3670
MECH-3671
MECH-4670
MECH-4671
MECH-4672
MECH-3430.	Automotive Engineering Fundamentals
MECH-4440.	Topics in Automotive Engineering
MECH-4441.	Directed Studies in Automotive Engineering
MECH-4463.	Vehicle Dynamics
MECH-4465.	Internal Combustion Engines
MECH-4467. 	Vehicle Thermal Management
MECH-4469


12

## 4. Clean and append all dataframes and write to CSV

In [27]:
# we only havent scraped the first 5 and the last one
course_names3 = course_names3[:5] + [course_names3[-1]]
course_codes3 = course_codes3[:5] + [course_codes3[-1]]
course_descs3 = course_descs3[:5] + [course_descs3[-1]]

In [28]:
df3 = pd.DataFrame({
    
    "Course Number": course_codes3,
    "Course Name": course_names3,
    "Course Description": course_descs3   
    
})

df3

,Course Number,Course Name,Course Description
0,MECH-3670,Aerospace Engineering Fundamentals,History of flight and aircraft evolution. Air...
1,MECH-3671,Aerospace Materials and Manufacturing,"Properties and selection of metals, ceramics, ..."
2,MECH-4670,Aerospace Propulsion,Application of gas dynamics and thermodynamics...
3,MECH-4671,Aerodynamics and Performance,Analysis of aircraft configurations. Viscous ...
4,MECH-4672,Flight Dynamics and Control of Unmanned Aerial...,Flight dynamics modelling for fixed-wing aircr...
5,MECH-4469,Diesel Engine Fundamentals,Theory and practice of modern diesel engines. ...


In [29]:
df= df.append(df2).append(df3, ignore_index=True)
df

,Course Number,Course Name,Course Description
0,MATH-1270,Linear Algebra,"This course will coverlinear systems,linear tr..."
1,MATH-1720,Differential Calculus,This course will cover trigonometric functions...
2,GENG-1110,Engineering Mechanics I,Statics of particles and rigid bodies; trusses...
3,GENG-1330,Engineering and Design,Introductory engineering design course. Visual...
4,GENG-1180,Engineering and the Profession,The Engineering and the Profession course is a...
...,...,...,...
66,MECH-3671,Aerospace Materials and Manufacturing,"Properties and selection of metals, ceramics, ..."
67,MECH-4670,Aerospace Propulsion,Application of gas dynamics and thermodynamics...
68,MECH-4671,Aerodynamics and Performance,Analysis of aircraft configurations. Viscous ...
69,MECH-4672,Flight Dynamics and Control of Unmanned Aerial...,Flight dynamics modelling for fixed-wing aircr...


In [31]:
df.to_csv('NEW_Windsor_MechEng_Core_and_Electives_(AllYears)_Courses.csv', index = False)

In [ ]:
#some manual fix: mech 4259 edit out the MECHANICAL: APPROVED COURSES... part
#geng 2200 remove course name at beginning of course description